In [1]:
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch

In [119]:
epochs = 10
range_min = -9
range_max = 9
batch_size = 4 
num_digits = 1
num_train_samples = 100
num_test_samples = 10

In [120]:
temp = np.random.randint(low = range_min,high = range_max+1,size = (num_train_samples,2),dtype = np.int64)
x_train = np.empty(shape = [num_train_samples,4])
x_train[:,:2] = temp
x_train[:,2] = [len(str(abs(x))) for x in temp[:,0]]
x_train[:,3] = [len(str(abs(x))) for x in temp[:,1]]
x_train[x_train[:,0] == 0] = np.random.randint(low=range_min,high=range_max+1)
y_train = np.empty(shape = [num_train_samples,2])
y_train[:,0] = np.array((np.abs(x_train[:,1]) / np.abs(x_train[:,0])))
y_train[:,1] = np.array(np.abs(x_train[:,1]) % np.abs(x_train[:,0]))

In [121]:
train_data = torch.from_numpy(x_train).type(torch.DoubleTensor)
train_labels = torch.from_numpy(y_train).type(torch.LongTensor)
# test_data = torch.from_numpy(x_test).type(torch.FloatTensor)
# test_labels = torch.from_numpy(y_test).type(torch.LongTensor)
torch.set_printoptions(precision = 10)
torch.set_default_dtype(torch.float64)

In [5]:
# This model adds two numbers
class Net_add(nn.Module):
    def __init__(self):
        super(Net_add,self).__init__()
        self.fc1 = nn.Linear(2,1,bias=False)
        self.fc1.weight.data = torch.tensor([[1.0,1.0]]).type(torch.float64)
    
    def forward(self,x):
        x = self.fc1(x)
        return x
    
    def predict(self,x):
        return self.forward(x).type(torch.float64)

# model_add = torch.load('model_add').double()
model_add = Net_add().type(torch.float64)

In [6]:
# This model subtracts two numbers
class Net_sub(nn.Module):
    def __init__(self):
        super(Net_sub,self).__init__()
        self.fc1 = nn.Linear(2,1,bias=False)
        self.fc1.weight.data = torch.tensor([[1,-1]]).type(torch.float64)
    
    def forward(self,x):
        x = self.fc1(x)
        return x
    
    def predict(self,x):
        return self.forward(x).type(torch.float64)
    
# model_sub = torch.load('model_sub').double()
# model_sub = torch.load('model_sub').type(torch.float64)
model_sub = Net_sub().type(torch.float64)

In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(2,30)
        self.fc2 = nn.Linear(30,82)
    
    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x))
        return x
    
    def predict(self,x):
        pred = self.forward(x)
        _,out = torch.max(pred,1)
        return out.type(torch.float64)

model_mul_0_9 = torch.load('mul_0_9').type(torch.float64)

In [8]:
# This model will give the absolute of a number
# input: 1 number
# output: absolute of that number 

class Net_abs(nn.Module):
    def __init__(self,model_add,model_sub):
        super(Net_abs,self).__init__()
        
        self.model_add = model_add
        self.model_sub = model_sub
        self.maxpool1d = nn.MaxPool1d(2)

    
    def forward(self,x):
        concat = torch.cat((x,x),1)
        add = self.model_add.predict(concat)
        sub = self.model_sub.predict(concat)
        
        concat = torch.cat((add,sub),1).unsqueeze(0)
        out = self.maxpool1d(concat).squeeze(0)
        
        concat = torch.cat((out,x),1)
        out = model_sub.predict(concat)
        
        return out
    
    def predict(self,x):
        return self.forward(x).type(torch.float64)
    
# model_abs = torch.load('model_abs').type(torch.float64)
model_abs = Net_abs(model_add,model_sub).type(torch.float64)

In [9]:
# This model will separate a number into digits
# input: 2 inputs - (number,num_digits)
# output: separated digits (num_digits,)  

class Net_separate(nn.Module):
    def __init__(self,model_sub):
        super(Net_separate,self).__init__()
        
        self.divide_by_10 = nn.Linear(1,1,bias=False).type(torch.float64)
        self.divide_by_10.weight.data = torch.tensor([[0.1]]).type(torch.float64)

        self.multiply_by_10 = nn.Linear(1,1,bias=False)
        self.multiply_by_10.weight.data = torch.tensor([[10]]).type(torch.float64)
        
        self.model_sub = model_sub
    
    def forward(self,x):
        input_number = x[:,0].unsqueeze(-1)
        num_digits = x[:,1].unsqueeze(-1)
        concat = []
        data = input_number
        for i in range(num_digits[0].type(torch.IntTensor)):
            temp_data = self.divide_by_10(data)
            temp_data = temp_data.floor()
            temp = self.multiply_by_10(temp_data)
            concat_temp = torch.cat((data,temp),1)
            digit = self.model_sub.predict(concat_temp)
            
            concat.insert(0,digit)
            data = temp_data
        
        out = torch.cat(concat,1)
        return out
    
    def predict(self,x):
        return self.forward(x).type(torch.float64)
    
# model_separate = torch.load('model_separate').double()  
# model_separate = torch.load('model_separate').type(torch.float64)
model_separate = Net_separate(model_sub).type(torch.float64)

In [10]:
# This model will give the product of two absolute numbers
# input: 2 separated numbers : (2,num_digits)
# output: product 

class Net_multiply_abs(nn.Module):
    def __init__(self,model_mul_0_9,model_add,model_separate):
        super(Net_multiply_abs,self).__init__()
        
        self.model_add = model_add
        self.model_separate = model_separate
        self.model_mul_0_9 = model_mul_0_9
        
        self.final_row_add_layers = []
    
    def forward(self,x):
        n = x.shape[-1]
        m = x.shape[-1]
        
        # get initial products
        products = []
        for i in range(x.shape[-1]):
            mul = x[:,0,i:i+1]
            for j in range(x.shape[-1]):
                temp = x[:,1,j:j+1]
                concat = torch.cat((temp,mul),1)
                prod = self.model_mul_0_9.predict(concat)
                products.insert(0,prod)
                
        # get individual rows
        row_values = []
        prod_index = 0
        two = torch.from_numpy(np.repeat(np.array([2.0],ndmin = 2),x.shape[0],axis = 0)).type(torch.float64)
        for r in range(n):
            for i in range(m+1):
                if i==0:
                    temp = torch.cat((products[prod_index].unsqueeze(-1),two),1)
                    sep = self.model_separate.predict(temp)
                    carry = sep[:,0:1]
                    ones_digit = sep[:,1:]
                    row_values.append(ones_digit)
                    prod_index+=1
                    
                elif i==m:
                    row_values.append(carry)
                else:
                    concat = torch.cat((products[prod_index].unsqueeze(-1),carry),1)
                    add = self.model_add(concat)
                    concat = torch.cat((add,two),1)
                    sep = self.model_separate.predict(concat)
                    carry = sep[:,0:1]
                    ones_digit = sep[:,1:]
                    row_values.append(ones_digit)
                    prod_index+=1
        
        final_row = []
        current_ind = 0 
        for i in range(m+n):
            if i ==0:
                final_row.insert(0,row_values[0])
                temp = torch.cat((row_values[0],two),1)
                sep = self.model_separate.predict(temp)
                carry = sep[:,0:1]
                current_ind+=1
            else:
                to_add = []
                ones = []
                to_add.append(row_values[current_ind])
                ones.append(1)
                to_add.append(carry)
                ones.append(1)
                
                next_ind = current_ind + m
                while(next_ind < n*(m+1) and next_ind%(m+1)!=0):
                    to_add.append(row_values[next_ind])
                    ones.append(1)
                    next_ind+=m
                if(next_ind < n * (m+1) and next_ind%(m+1)==0):
                    to_add.append(row_values[next_ind])
                    ones.append(1)

                self.final_row_add_layers.append(nn.Linear(len(to_add),1,bias=False))
                self.final_row_add_layers[-1].weight.data = torch.tensor([ones]).type(torch.float64)
                concat = torch.cat(to_add,-1)
                add = self.final_row_add_layers[-1](concat)
                
                temp = torch.cat((add,two),1)
                sep = self.model_separate.predict(temp)
                carry = sep[:,0:1]
                ones_digit = sep[:,1:]
                final_row.insert(0,ones_digit)
                
                if (current_ind + 1) % (m+1) !=0:
                    current_ind+=1
                else:
                    current_ind+=m+1
                
                
        out = torch.cat(final_row,-1)
        return out
    
    def predict(self,x):
        return self.forward(x).type(torch.float64)
    
# model_multiply_abs = torch.load('model_multiply_abs').double()
model_multiply_abs = Net_multiply_abs(model_mul_0_9,model_add,model_separate).type(torch.float64)

In [11]:
# This model will join a list of digits to a number
# input: 1 input - (num_digits,)
# output: 1   

class Net_merge(nn.Module):
    def __init__(self):
        super(Net_merge,self).__init__()
        self.layers = []
        
    def forward(self,x):
        n = x.shape[-1]
        i = 0 
        add_list = []
        ones = []
        while(n>=1):
            temp = x[:,n-1:n]
            self.layers.append(nn.Linear(1,1,bias=False))
            self.layers[-1].weight.data = torch.tensor([[10**i]]).type(torch.float64)
            out = self.layers[-1](temp)
            i +=1
            n-=1
            add_list.append(out)
            ones.append(1.0)
        
        self.layers.append(nn.Linear(x.shape[-1],1,bias=False))
        self.layers[-1].weight.data = torch.tensor([ones]).type(torch.float64)
        concat = torch.cat(add_list,-1)
        out = self.layers[-1](concat)
        return out
    
    def predict(self,x):
        return self.forward(x).type(torch.float64)

# model_merge = torch.load('model_merge').double()
model_merge = Net_merge().type(torch.float64)

In [12]:
# this model mulltiplies a single digit number with a sign (1,-1)
# input: 2 numbers (2,)
# output: single number 
class Net_mul_sign_0_9(nn.Module):
    def __init__(self):
        super(Net_mul_sign_0_9,self).__init__()
        self.fc1 = nn.Linear(2,10)
        self.fc2 = nn.Linear(10,1)
    
    def forward(self,x):
        x = F.softsign(self.fc1(x))
        x = self.fc2(x)
        return x
    
    def predict(self,x):
        out = self.forward(x)
        return torch.round(out).type(torch.float64)

model_mul_sign_0_9 = torch.load('model_mul_sign_0_9').type(torch.float64)

In [13]:
# this model mulltiplies a number with a sign (1,-1)
# input: 3 numbers (3,) - [number,sign,num_digits]
# output: single number 
class Net_mul_sign_abs(nn.Module):
    def __init__(self,model_separate,model_mul_sign_0_9):
        super(Net_mul_sign_abs,self).__init__()
        self.model_separate = model_separate
        self.model_mul_sign_0_9 = model_mul_sign_0_9
        self.multiply_2 = nn.Linear(1,1,bias = False)
        self.multiply_2.weight.data = torch.tensor([[2.0]]).type(torch.float64)
        self.layers = []
    
    def forward(self,x):
        number = x[:,0].unsqueeze(-1)
        sign = x[:,1].unsqueeze(-1)
        num_digits = x[:,2].unsqueeze(-1)
        
        output_digits = self.multiply_2(num_digits)
        concat = torch.cat((number,output_digits),1)
        sep = self.model_separate.predict(concat)
        products = []
        i = 0
        j = sep.shape[-1]
        for n in range(j,0,-1):
            temp = sep[:,n-1:n]
            concat = torch.cat((temp,sign),1)
            prod = self.model_mul_sign_0_9.predict(concat)
            self.layers.append(nn.Linear(1,1,bias=False))
            self.layers[-1].weight.data = torch.tensor([[10**i]]).type(torch.float64)
            prod = self.layers[-1](prod)
            products.append(prod)
            i+=1
        
        concat = torch.cat(products,1)
        self.layers.append(nn.Linear(1,1,bias=False))
        self.layers[-1].weight.data = np.repeat(torch.tensor([[1.0]]),len(products),axis = -1).type(torch.float64)
        out = self.layers[-1](concat)
        return out
    
    def predict(self,x):
        out = self.forward(x)
        return out.type(torch.float64)
    
model_mul_sign_abs = Net_mul_sign_abs(model_separate,model_mul_sign_0_9).type(torch.float64)

In [14]:
class Net_separate_sign(nn.Module):
    def __init__(self):
        super(Net_separate_sign,self).__init__()
        self.fc1 = nn.Linear(1,1,bias=False)
        self.fc1.weight.data = torch.tensor([[1.01]]).type(torch.float64)
        
    def forward(self,x):
        out = F.softsign(self.fc1(x))
        return out
    
    def predict(self,x):
        return self.forward(x).round().type(torch.float64)

model_separate_sign = Net_separate_sign().type(torch.float64)

In [15]:
class Net_mul_signs(nn.Module):
    def __init__(self,model_add,model_sub,model_abs):
        super(Net_mul_signs,self).__init__()
        self.model_add = model_add
        self.model_abs = model_abs
        self.model_sub = model_sub
        
    def forward(self,x):
        out = self.model_add.predict(x)
        out = self.model_abs.predict(out)
        
        ones = torch.ones(out.shape).type(torch.float64)
        out = torch.cat((out,ones),1)
        out = self.model_sub.predict(out)
        
    
        return out
    
    def predict(self,x):
        return self.forward(x).type(torch.float64)

model_mul_signs = Net_mul_signs(model_add,model_sub,model_abs).type(torch.float64)

In [16]:
# this model mulltiplies two numbers
# input: 3 numbers (3,) - [number,number,num_digits]
# output: single number 
class Net_multiply(nn.Module):
    def __init__(self,model_separate_sign,model_mul_signs,model_abs,model_separate,model_multiply_abs,model_merge,model_mul_sign_abs):
        super(Net_multiply,self).__init__()
        self.model_separate_sign = model_separate_sign
        self.model_mul_signs = model_mul_signs
        self.model_abs = model_abs
        self.model_separate = model_separate
        self.model_multiply_abs = model_multiply_abs
        self.model_merge = model_merge
        self.model_mul_sign_abs = model_mul_sign_abs
    
    def forward(self,x):
        numbers = x[:,:2]
        num_digits = x[:,2:]
        
        # get sign
        sign1 = self.model_separate_sign.predict(x[:,0:1])
        sign2 = self.model_separate_sign.predict(x[:,1:2])
        concat = torch.cat((sign1,sign2),1)
        sign = self.model_mul_signs.predict(concat)
        
        # get absolutes
        abs_inp1 = self.model_abs.predict(x[:,0:1])
        abs_inp2 = self.model_abs.predict(x[:,1:2])
        
        # separate absolutes
        concat = torch.cat((abs_inp1,num_digits),1)
        sep_1 = self.model_separate.predict(concat).unsqueeze(1)
        concat = torch.cat((abs_inp2,num_digits),1)
        sep_2 = self.model_separate.predict(concat).unsqueeze(1)
        
        # multiply absolutes
        concat = torch.cat((sep_1,sep_2),1)
        abs_prod = self.model_multiply_abs(concat)
        
        # merge absolute product
        merged_abs_prod = self.model_merge(abs_prod)
        # multiply sign and absolute product
        concat = torch.cat((merged_abs_prod,sign,num_digits),1)
        out = self.model_mul_sign_abs.predict(concat)
        
        return out
    
    def predict(self,x):
        out = self.forward(x)
        return out.type(torch.float64)
    
model_multiply = Net_multiply(model_separate_sign,model_mul_signs,model_abs,model_separate,model_multiply_abs,model_merge,model_mul_sign_abs).type(torch.float64)

In [105]:
# this model mulltiplies two numbers
# input: 3 numbers (3,) - [number,number,num_digits]
# output: single number 
class Net_divide(nn.Module):
    def __init__(self,model_multiply,model_separate,model_sub,model_merge,model_add,model_separate_sign,model_mul_signs,model_abs,model_mul_sign_abs):
        super(Net_divide,self).__init__()
        self.model_multiply = model_multiply
        self.model_separate = model_separate
        self.model_sub = model_sub
        self.model_merge = model_merge
        self.model_add = model_add
        self.model_separate_sign = model_separate_sign
        self.model_mul_signs = model_mul_signs
        self.model_abs = model_abs
        self.model_mul_sign_abs = model_mul_sign_abs
        
        self.maxpool1d = nn.MaxPool1d(2)
        self.fc1 = nn.Linear(1,1,bias=False)
        self.fc1.weight.data = torch.tensor([[-1.0]]).type(torch.float64)
        self.maxpool1d_2 = nn.MaxPool1d(10)
        self.fc2 = nn.Linear(1,1,bias=False)
        self.fc2.weight.data =  torch.tensor([[10]]).type(torch.float64)
    
    def forward(self,x):
        divisor = x[:,0:1]
        dividend = x[:,1:2]
        length_of_divisor = x[:,2:3]
        length_of_dividend = x[:,3:]
        
        # get sign
        sign1 = self.model_separate_sign.predict(divisor)
        sign2 = self.model_separate_sign.predict(dividend)
        concat = torch.cat((sign1,sign2),1)
        sign = self.model_mul_signs.predict(concat)
        
        
        divisor = self.model_abs.predict(divisor)
        dividend = self.model_abs.predict(dividend)
        
        X = []
        for i in range (0,10):
            temp = np.repeat(torch.tensor([[i]]),x.shape[0],axis = 0).type(torch.float64)
            concat = torch.cat((temp,divisor,length_of_divisor),1)
            answer = self.model_multiply.predict(concat)
            X.append(answer)
        
        n = length_of_divisor.type(torch.IntTensor)
        concat = torch.cat((dividend,length_of_dividend),1)
        sep_dividend = self.model_separate.predict(concat)
        quotient = [0]
        remainder = torch.tensor([[0]]).type(torch.float64)
        i = 0
        ind_start = 0
        ind_end = 1
        while(i < length_of_dividend[0].type(torch.IntTensor) ) :
            chunk = sep_dividend[:,ind_start:ind_end]
            num_digits_chunk = torch.tensor([chunk.shape[-1]]).unsqueeze(-1).type(torch.float64)
            chunk = self.model_merge.predict(chunk)
            temp = self.fc2(remainder)
            concat = torch.cat((temp,chunk),1)
            chunk = self.model_add.predict(concat)
            sub = []
            for j in range(len(X)):
                concat = torch.cat((chunk,X[j]),1)
                temp = self.model_sub.predict(concat)
                sub.append(temp)
            sub_prods = []
            ones = torch.ones(length_of_divisor.shape)
            concat = torch.cat((length_of_divisor,ones),1)
            num_digits_temp = self.model_add.predict(concat)
            for j in range(len(sub)):
                concat = torch.cat((sub[j],sub[-1],num_digits_temp),1)
                temp = self.model_multiply.predict(concat)
                sub_prods.append(temp)
            
            max_pools = []
            for j in range(len(sub)):
                concat = torch.cat((sub[j],sub_prods[j]),1).unsqueeze(0)
                temp = self.maxpool1d(concat).squeeze(0)
                temp = self.fc1(temp)
                max_pools.append(temp)
                
            max_pools = torch.tensor([max_pools])
            _,q = torch.max(max_pools,1)
            quotient.append(q)
            
            remainder = self.maxpool1d_2(max_pools.unsqueeze(0)).squeeze(0)
            remainder = self.fc1(remainder)
            ind_start = ind_end
            ind_end = ind_start+1
            
            i+=1
            
        out = torch.tensor(quotient).type(torch.float64).unsqueeze(0)
        q = self.model_merge.predict(out)
        temp = torch.tensor([[out.shape[-1]]]).type(torch.float64)
        concat = torch.cat((q,sign,temp),1)
        out = self.model_mul_sign_abs.predict(concat)
        out = torch.cat((out,remainder),1)
        return out

In [125]:
model_divide = Net_divide(model_multiply,model_separate,model_sub,model_merge,model_add,model_separate_sign,model_mul_signs,model_abs,model_mul_sign_abs).type(torch.float64)
# d = torch.tensor([[ 9, 98,  1,  2]]).type(torch.float64)
n = 13
print(train_data[n:n+1,:])
print(train_labels[n:n+1])
model_divide.forward(train_data[n:n+1])

tensor([[-1.,  8.,  1.,  1.]])
tensor([[8, 0]])


C:\Users\Ammar\.conda\envs\neuralnets\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if __name__ == '__main__':


tensor([[-8.,  0.]], grad_fn=<CatBackward>)

In [122]:
for j in range(1,train_data.shape[0]):
    out = model_divide.forward(train_data[j-1:j])
    temp = train_labels[j-1].unsqueeze(0).numpy()
    out = out.detach().numpy()
    correct = 0
    if out[0,0] == temp[0,0] and out[0,1] == temp[0,1]:
        correct+=1
        print('correct')
    else:
        print('wrong example :',j)
print(correct)

C:\Users\Ammar\.conda\envs\neuralnets\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if __name__ == '__main__':


correct
correct
correct
correct
correct
correct
correct
wrong example : 8
correct
correct
correct
correct
correct
wrong example : 14
correct
correct
correct
correct
correct
correct
wrong example : 21
wrong example : 22
correct
correct
wrong example : 25
correct
wrong example : 27
correct
correct
correct
correct
correct
correct
correct
correct
wrong example : 36
correct
correct


KeyboardInterrupt: 